# AC Hash Cons
Take microegg, replace AC with just a hash cons
https://www.philipzucker.com/weighted_uf/ do this with lpo? just store nxn lpo results?
Well, depth can short circuit it a little and max symbol
There is also ACKBO

The +{x} singleton kind of has an idempotence thing to it.
Pattern matching on that would branch on stripping it or not.


It'd be better to return the first bin and a generator of the remaining bins for early pruning. Whatever


In [13]:
import itertools
def partitions(s, n):
    for assign in itertools.product(range(n), repeat=len(s)):
        bins = [[] for _ in range(n)]
        for x,b in zip(s, assign):
            bins[b].append(x)
        # possibly filter on nonempty or singleton or do map into ACId here
        yield tuple(tuple(bin) for bin in bins)

list(partitions("abc", 2))

[(('a', 'b', 'c'), ()),
 (('a', 'b'), ('c',)),
 (('a', 'c'), ('b',)),
 (('a',), ('b', 'c')),
 (('b', 'c'), ('a',)),
 (('b',), ('a', 'c')),
 (('c',), ('a', 'b')),
 ((), ('a', 'b', 'c'))]

In [ ]:
from dataclasses import dataclass,field
import itertools
type Id = int

@dataclass(frozen=True)
class ACId:
    f : str
    args : tuple[Id, ...]

type SId = Id | ACId # structured Ids. ACApp may be uninterned / ephemeral
# obviously it hurts to keep making copies of the same struct, but they are noticeably different

@dataclass(frozen=True)
class App:
    f : str
    args : tuple[Id, ...]

@dataclass(frozen=True)
class ACApp:
    f : str
    args : tuple[Id, ...]
type Node = App | ACApp

@dataclass(frozen=True)
class Var:
    name : str

@dataclass(frozen=True)
class PApp:
    f : str
    args : tuple['Pattern', ...]

@dataclass(frozen=True)
class PACApp:
    f : str
    args : tuple['Pattern', ...]

type Pattern = Var | PApp | PACApp

@dataclass
class ACHashCons:
    memo : dict[Node, Id] = field(default_factory=dict) # IndexMap
    nodes : dict[Id, Node] = field(default_factory=dict)

    def mk_node(self, node : Node) -> Id:
        if node in self.memo:
            return self.memo[node]
        else:
            id = len(self.nodes)
            self.nodes[id] = node
            self.memo[node] = id
            return id

    def mk_app(self, f : str, args : tuple[SId, ...]) -> Id:
        args = tuple(self._mk_acapp(arg.f, arg.args) if isinstance(arg, ACId) else arg for arg in args)
        return self.mk_node(App(f, args))

    # maybe you should never make an ac_app directory, only mk_app should call it? Is this overcomplicating it?
    #def mk_acapp(self, f : str, args : tuple[SId, ...]) -> SId:
        #args = tuple(args if isinstance(arg, int) else self._mk_acapp(arg.f, arg.args) for arg in args)
        # should also flatten
        # Should it flatten by searching into nodes too? No i think
        #flatargs = []
    #    return ACId(f, args)

    def mk_acapp(self, f : str, args : tuple[Id, ...]) -> Id:
        flatargs = [] 
        for id in args:
            assert isinstance(id, int)
            n = self.nodes[id]
            if isinstance(n, ACApp) and n.f == f:
                flatargs.extend(n.args)
            else:
                flatargs.append(id)
        flatargs.sort()
        return self.mk_node(ACApp(f, tuple(flatargs)))

    def acmatch_rec(self, p : Pattern, id : SId, subst) -> list[dict[str, Id]]:
        if isinstance(id, ACId):
            n = id
        else:
            n = self.nodes[id]
            if isinstance(n, ACApp):
                n = ACId(n.f, n.args) # could move this below Var for efficiency, but whatev
        match p, n:
            case Var(x), _:
                if x in subst:
                    if subst[x] == id:
                        return [subst]
                    else:
                        return []
                else:
                    newsubst = subst.copy()
                    newsubst[x] = id
                    return [newsubst]
            case PApp(pf, pargs), App(f, args) if pf == f and len(pargs) == len(args):
                substs = [subst]
                for parg, arg in zip(pargs, args):
                    substs = [subst for subst0 in substs for subst in self.acmatch_rec(parg, arg, subst0)]
                return substs
            case PACApp(pf, pargs), ACId(f, args) if pf == f:
                # we need to hand down temporary ACNodes
                # What about ACNode with 1 element? ACI easier?
                res = []
                for subargs in partitions(args, len(pargs)):
                    subargs = map(lambda a: ACId(f, a), subargs)
                    substs = [subst]
                    for parg, arg in zip(pargs, subargs):
                        substs = [subst for subst0 in substs for subst in self.acmatch_rec(parg, arg, subst0)]
                    res.extend(substs)
                return res
            case _:
                return []

        
    def acmatch(self, p : Pattern, id : Id) -> list[dict[str, Id]]:
        # flatten pattern?
        return self.acmatch_rec(p, id, {})

#for partition in itertools.combinations(args, len(pargs)): # wrong
#    for subargs in itertools.permutations(partition

hc = ACHashCons()
a = hc.mk_app("a", ())
b = hc.mk_app("b", ())
c = hc.mk_app("c", ())
pabc = hc.mk_acapp("+", (a, b, c))
fa  = hc.mk_app("f", (a,))
ffa = hc.mk_app("f", (fa,))

hc.acmatch(PACApp("+", (Var("x"), Var("y"))), pabc)
hc.acmatch(PACApp("+", (Var("x"), PACApp("+", (Var("z"), Var("y"))))), pabc)
hc.acmatch(PACApp("+", (Var("x"), PACApp("+", (Var("x"), Var("y"))))), pabc)
hc.acmatch(PApp("f", (Var("x"),)), ffa)


[{'x': 4}]


Post hoc equalities may add new flattenings, so rebuilding involve AC flattening to make sure all AC flat forms exists.
Rebuilding becomes way more expensive
The Aegraph style of eagerly applying rewrites may avoid needing as much rebuilding though.

Rudi points out, what about right associated sorted regular hash consing. What about KBO on that?


Multisets of multiset rewriting. Take the  https://www.philipzucker.com/multiset_rw/
Nested multisets
Non well founded multisets? The AC to the ACI of non well founded sets

https://pdfs.semanticscholar.org/08a6/9eb42f087ce76876aef224d32f2109367307.pdf modular mutliset rewriting


# ACKBO
http://cl-informatik.uibk.ac.at/software/ackbo/
https://arxiv.org/abs/1403.0406


Ok the simplest ordering they describe puts all AC simple low, so they want to get pused to leaved

# ACRPO
I still want to implement acrpo. 

https://research.vu.nl/ws/portalfiles/portal/266879340/A_Lambda_Free_Higher_Order_Recursive_Path_Order.pdf implement lfrpo

Ground nominal term orderings.

Equivariance means that X = Y can't be oriented. or add(X,Y) = add(Y,X)
But we're ground so we're probably better behaved

Rudi noted that slots are lot of like unification vars that never get narrowed and have a disjointness constraint.
https://egraphs.zulipchat.com/#narrow/channel/328972-general/topic/EGRAPHS.20Meeting.202025-03-20.3A.20Slotted.20E-Graphs/near/543056830

If `t =g= s` was useful, what about `t <g< s`. Hmm. Ordered modulo g. Then we could talk about x + y < y + x ?

`2 <(-7)< -4` using shift group. Yes...  also `2 <-8< -4` but not `2 <-6< -4`
Combininig the inequality union find with the group union find.

An ordered group? https://en.wikipedia.org/wiki/Linearly_ordered_group
https://ncatlab.org/nlab/show/ordered+group
https://en.wikipedia.org/wiki/Partially_ordered_group

`def perm_lpo(t, s) -> tuple[Order, G]:`


https://memoryleak47.github.io/slotted-egraphs-demystified-i/
https://medium.com/@mineralsteins/ultimate-code-compression-with-slotted-egraph-74f1859c9df5

So we need an unfailing variant of knuth bendix

renamings aren't bijective. They are 1-1.

https://www.cs.ru.nl/~cynthiakop/2024_isr/day2handout.pdf  Termination:
the higher-order recursive path ordering - Kop


Have I done dumb ground kb in python?

https://drops.dagstuhl.de/storage/00lipics/lipics-vol021-rta2013/LIPIcs.RTA.2013.319/LIPIcs.RTA.2013.319.pdf normalized completion revisited. 2013. Has a table of moidifed rules. hmm. acrpo better than ackbo?


https://www.jonmsterling.com/01JQ/ disentangling unfication and implciti coercions 
"nested pattern unification" https://andraskovacs.github.io/pdfs/wits23prez.pdf



In [ ]:
class ACEGraph()
    def add_ac(self, f, args):
        flatargs = [[]]
        for id in args:
            this_arg = [[id]]
            for node in self.class[id]:
                match node:
                    case AC(f1, args1) if f1 == f:
                        this_arg.append(args1)
                    case _:
                        pass
            # expand out all ways of flattening
            flatargs = [prev + this for prev in flatargs for this in this_arg]
        children = []
        for args in flatargs: # maybe dedup first. Or whatever. add_prim will do that
            seld.add_prim(AC(f, sorted(args)))

        if AC(f, sorted(args)) in memo:
            return memo
        else:
            # do eager flattening
            # add in all E equivalent flattened terms
            for sub in powerset(args)
                if AC(f, sub) in memo:
                    id = mem(AC(f, sub))
                    AC
    def rebuild():
        add in flattenings
        
    def ematch(pattern, id)::
        for node in eclasses[id]:
            match node:
                case App
                case ACApp:
                    for all partitions:
                        match

In [ ]:
from dataclasses import dataclass
@dataclass(frozen=True)
class App():
    f : str
    args : tuple["Term", ...] = ()

    def __len__(self) -> int:
        return 1 + sum(len(arg) for arg in self.args)

    def __repr__(self) -> str:
        if self.args:
            return f"{self.f}({', '.join(repr(arg) for arg in self.args)})"
        else:
            return self.f

    def __le__(self, other: "App") -> bool:
        return self == other or self < other
    def __lt__(self, other: "App") -> bool:
        if self == other:
            return False
        ls,lo = len(self), len(other)
        if ls != lo:
            return ls < lo
        if self.f != other.f:
            return self.f < other.f
        for sa, oa in zip(self.args, other.args):
            if sa != oa:
                return sa < oa
        return False
    def subst(self, a, b):
        if self == a:
            return b
        return App(self.f, tuple(arg.subst(a,b) for arg in self.args))    

x = App("x")
y = App("y")
f = lambda a,b: App("f", (a,b))
assert f(x,y) > x
assert y > x
assert not x > y
assert not x < x

@dataclass
class KB():
    E : list[tuple[App,App]]
    R : list[tuple[App,App]]
    # Ordering

    def orient(self, n: int):
        (a,b) = self.E.pop(n)
        assert b != a
        if b > a:
            a,b = b,a
        self.R.append((a,b))
        return self
    
    def triv(self, n : int):
        (a,b) = self.E.pop(n)
        assert a == b
        return self
    
    def simp(self, en : int, rn : int):
        (ae,be) = self.E.pop(en)
        (ar,br) = self.R[rn]
        self.E.append((ae.subst(ar,br), be.subst(ar,br)))
        return self
    
    def deduce(self, u : App, rleft : int, rright: int):
        (arl,brl) = self.R[rleft]
        (arr,brr) = self.R[rright]
        self.E.append((u.subst(arl,brl),
                       u.subst(arr,brr)))
        return self
    
    def rsimp(self, rn : int, rm : int):
        (ar,br) = self.R[rn]
        (am,bm) = self.R.pop(rm)
        self.R.append((am, bm.subst(ar,br)))
        return self

    def lsimp(self, rn : int):
        # TODO: need to dignify with encompassment condition
        self.E.append(self.R.pop(rn))
        return self



kb = KB([(f(x,y), y), (f(x,y), x)], [])

kb
kb.orient(0)
kb.simp(0,0)
kb
kb.orient(0)
kb.rsimp(1,0)
kb.deduce(f(x,y), 0, 1)
kb.orient(0)
kb.lsimp(1)
kb.simp(0,0)
kb.simp(0,1)
kb.triv(0)

KB(E=[], R=[(y, x), (f(x, x), x)])

In [ ]:
# ok but it hsould be schedules

In [ ]:
from dataclasses import dataclass, field

counter = 0
def fresh_counter() -> int:
    global counter
    counter += 1
    return counter

@dataclass(frozen=True)
class Slot():
    name : int = field(default_factory=fresh_counter)

@dataclass(frozen=True)
class App():
    f : str
    args : tuple["Term", ...] = ()

type Term = App | Slot
